In [1]:
from gamearg import *
import clingo

In [2]:
def generate_predicate_strings(df, pw_keyword):
    try:
        predicate_strings = []
        for _, row in df.iterrows():
            node_value = row['node']
            node_status = row [pw_keyword]
            predicate_string = f':- not len("{node_status}",{node_value},_).'
            predicate_strings.append(predicate_string)
        return predicate_strings
    except:
        raise ValueError(f"{pw_keyword} does not exist in the DataFrame")


def get_min_changes(input_file, pw_keyword, keyword, reverse):
    wfs_stb_pws, df_wfs_stb=node_stb_cal(input_file, keyword, reverse)
    enforce_log="""
    node(X) :- move(X,_).
    node(Y) :- move(_,Y).
    
    {del(X,Y)}:- move(X,Y).
    
    edge(X,Y):- move(X,Y), not del(X,Y).
    
    % ensure that the graph with removed edge is a connected graph
    e(X,Y):- edge(X,Y).
    e(X,Y):- edge(Y,X).
    connected(X,Y) :- e(X,Y).
    connected(X,Y) :- connected(X,Z), e(Z,Y).
    :- node(X), node(Y), not connected(X,Y), X!=Y.
    
    % calculate the draw, lost, won for the new graph
    #const state_max = 100. 
    win_o(S, X) :- edge(X,Y), not win_u(S,Y), next(S,_). 
    win_u(S1,X) :- edge(X,Y), not win_o(S,Y), next(S,S1).
    fg(S1,X) :- next(S,S1), not win_u(S,X), win_u(S1,X).
    fr(0,X)  :- node(X), not win_o(0,X). 
    fr(S1,X) :- next(S,S1), not final(S), win_o(S,X), not win_o(S1,X). next(0,1). 
    next(S,S1) :- fg(S,X), S1 = S+1, S < state_max.
    final(S) :- next(S,S1), not next(S1,_). 
    len("lost",X,L)   :- fr(S,X), L = 2*S.   
    len("won",X,L)    :- fg(S,X), L = 2*S-1. 
    len("draw",X,"∞") :- node(X), not len("lost",X,_), not len("won",X,_). 
    
    % ensure no draw appears in the new graph
    :- len("draw",X,"∞").
    
    """+"\n".join(generate_predicate_strings(df_wfs_stb,pw_keyword)) +"\n\n#show del/2."
    ctl = clingo.Control()
    ctl.configuration.solve.models = "0"
    ctl.load(input_file)
    ctl.add("base", [], enforce_log)
    ctl.ground([("base", [])])
    output = []
    ctl.solve(on_model=lambda m: output.append(m.symbols(shown=True)))
    return(min(output, key=len))

In [3]:
print(get_min_changes("files/double_loop.lp", "pw_1", keyword="game", reverse=False))

[del(b,a), del(b,c)]
